In [1]:
import requests
import re
from dateutil.relativedelta import relativedelta
from datetime import datetime
from datetime import timedelta
from datetime import date
import pandas as pd
import time
import numpy as np
from bs4 import BeautifulSoup as bs

In [2]:
!ls

2017_salaries.csv                    boxscores2020.csv
2018_salaries.csv                    boxscores_2017.csv
2019_salaries.csv                    boxscores_2018.csv
2020_salaries.csv                    boxscores_2019.csv
Actual Name transform.ipynb          boxscores_2020.csv
Defensive stats.xlsx                 full_df.p
Get boxscores (3).ipynb              full_df_2017.p
Get_games (step 2).ipynb             full_df_2018.p
Get_teams (step 1).ipynb             full_df_2019.p
Player_list_2017.csv                 full_df_2020.p
Salary scrape.ipynb                  games2017.csv
Starting analysis.ipynb              games2018.csv
Untitled.ipynb                       games2019.csv
all_data.csv                         notes from youtube video on bs.ipynb
all_records.p                        records2017.csv
all_records_2017.p                   records2018.csv
all_records_2018.p                   records2019.csv
all_records_2019.p                   save_for_later_2019_scores.xlsx
all_records

In [3]:
df = pd.read_csv('2020_salaries.csv')

# NEW VERSION

In [6]:
#response = get_site(date(2019,10,31))
#text = get_text(response)
#soup_ = get_soup(text)
#data = get_data(soup_)
#trs = get_rows(soup_)


In [7]:
#trs[0]

In [8]:
#tds = trs[-1].find_all('td')

In [9]:
#'0' in [j.text for j in tds]

In [10]:
def get_minutes(trs):
    all_mins = []
    for i in trs:
        tds = i.find_all('td')
        try:
            minutes = [j.text for j in tds if re.fullmatch(r'\d+:\d+',j.text)][0]
            all_mins.append(minutes)
        except:
            if '0' in [j.text for j in tds]:
                minutes = 0
                all_mins.append(minutes)
            else:
                pass
            
        
    return pd.Series(all_mins,name='minutes')

In [12]:
#[i for i in get_minutes(trs)]

In [13]:
#[i.text for i in trs[2].find_all('td')]

In [14]:
#trs[0].text

In [16]:
#home_away(data[0])

In [17]:
#re.findall(r'[@v] [a-z][a-z][a-z]',data[1])

In [18]:
def get_site(date):
    response = requests.get(f'http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon={date.month}&day={date.day}&year={date.year}')
    return response

def get_text(response):
    return response.text

def get_soup(text):
    return bs(text)
    
def get_data(soup_):
    the_text = soup_.text
    first = the_text.index("Jump to")
    second = the_text.index("RotoGuru is produced by Dave Hall")
    all_we_need = the_text[first:second]
    #just pulling out relevant rows
    return [i for i in all_we_need.split("\n") if "Jump to" not in i and i != '']

#minutes and probably better way to get remainder of data
def get_rows(soup_):
    trs = soup_.find_all('table',attrs={'cellspacing':5})[0].find_all('tr')[2:]
    return trs

def get_minutes(trs):
    all_mins = []
    for i in trs:
        tds = i.find_all('td')
        try:
            minutes = [j.text for j in tds if re.fullmatch(r'\d+:\d+',j.text)][0]
            all_mins.append(minutes)
        except:
            if '0' in [j.text for j in tds]:
                minutes = 0
                all_mins.append(minutes)
            else:
                pass
            
        
    return pd.Series(all_mins,name='minutes')


    
#position
def position(row):
    if row[0] == 'C':
        return 'C'
    else:
        return row[:2]
    
#names
def name(row):
    if row[0] == 'C':
        return re.findall(r'[\'\s\-\.a-zA-Z]*, [\'\s\-\.a-zA-Z]*',row)[0][1:]
    else:
        return re.findall(r'[\'\s\-\.a-zA-Z]*, [\'\s\-\.a-zA-Z]*',row)[0][2:]
    

def salary(row):
    if row[:2] == 'NA':
        return np.nan
    return re.findall(r'\$\d*,\d*',row)[0]


def team(row):
    return re.findall(r'[a-z][a-z][a-z][@v]',row)[-1][:-1]
    #if '@' in row:
    #    return re.findall('00+[a-z]*',row)[0][-3:]
    #else:
    #    return re.findall('00+[a-z]*',row)[0][-4:-1]

def opponent(row):
    return re.findall(r'[@v] [a-z][a-z][a-z]',row)[-1][-3:]

def home_away(row):
    string_ = re.findall(r'[@v] [a-z][a-z][a-z]',row)
    if string_[0][0] == 'v':
        return 'home'
    else:
        return 'away'

def stats(row,stat):
    try:
        return re.findall(r'[0-9\-]*(?={0})'.format(stat),row)[0]
    except:
        return np.nan
    
def points(row):
    return stats(row,'pt')
    
def rebounds(row):
    return stats(row,'rb')
    
def assists(row):
    return stats(row,'as')
    
def steals(row):
    return stats(row,'st')

def blocks(row):
    return stats(row,'bl')

def turnovers(row):
    return stats(row,'to ')

def threes(row):
    return stats(row,'trey')

def fieldgoal(row):
    return stats(row,'fg')

def freethrow(row):
    return stats(row,'ft')
    
def get_df(dt):
    response = get_site(dt)
    text = get_text(response)
    soup_ = get_soup(text)
    data = get_data(soup_)
    positions = pd.Series(map(position,data),name = 'position')
    #stats
    points_ = pd.Series(map(points,data),name = 'points')
    rebounds_ = pd.Series(map(rebounds,data),name = 'rebounds')
    assists_ = pd.Series(map(assists,data),name = 'assists')
    steals_ = pd.Series(map(steals,data),name = 'steals')
    blocks_ = pd.Series(map(blocks,data),name = 'blocks')
    turnovers_ = pd.Series(map(turnovers,data),name = 'tos')
    threes_ = pd.Series(map(threes,data),name = 'threes')
    fieldgoal_ = pd.Series(map(fieldgoal,data),name = 'fg')
    freethrow_ = pd.Series(map(freethrow,data),name = 'ft')
    #minutes differently
    trs = get_rows(soup_)
    minutes = get_minutes(trs)
    
    names = pd.Series(map(name,data), name = 'name')
    salaries = pd.Series(map(salary,data),name = 'salary')
    teams = pd.Series(map(team,data),name = 'team')
    opponents = pd.Series(map(opponent,data),name = 'opponent')
    home_away_ = pd.Series(map(home_away,data),name = 'home_away')
    dates = pd.Series([dt]*len(names),name = 'date')
    df = pd.concat([names,positions,salaries,teams,\
                    points_,rebounds_,assists_,steals_,\
                    blocks_,turnovers_,threes_,fieldgoal_,\
                    freethrow_,minutes,dates,opponents, home_away_],axis=1)
    return df


def all_dates(start_date, end_date, verbose=False, to_csv = False, print_errors = False):
    date_range = pd.date_range(start_date, end_date)
    all_salaries = pd.DataFrame()
    for i in date_range:
        if verbose:
            print(i)
        try:
            df = get_df(i)
        except Exception as e:
            if print_errors:
                print(i,e)
            df = pd.DataFrame()
        all_salaries = all_salaries.append(df)
        
    if to_csv:
        all_salaries.to_csv(f'~/Desktop/{i.year}_salaries.csv',index=False)
    print('complete')
    
    
    return all_salaries

In [20]:
from datetime import timedelta

In [22]:
start = date(2020,12,1)
end = date.today()+timedelta(days=1)

all_sals = all_dates(start,end,verbose=False,to_csv=True,print_errors=False)

complete


In [37]:
all_sals

,name,position,salary,team,points,rebounds,assists,steals,blocks,tos,threes,fg,ft,minutes,date,opponent,home_away
0,"Ayton, Deandre",NA,NaN,pho,21.0,9,1,4,NaN,2,NaN,8-10,5-5,24:32,2020-12-16,lal,home
1,"Drummond, Andre",NA,NaN,cle,18.0,14,5,NaN,NaN,2,1,7-16,3-4,20:43,2020-12-16,nyk,away
2,"LaVine, Zach",NA,NaN,chi,24.0,3,5,1,1,1,3,9-14,3-3,25:22,2020-12-16,okc,away
3,"Robinson, Mitchell",NA,NaN,nyk,13.0,10,1,NaN,4,NaN,NaN,5-7,3-4,31:51,2020-12-16,cle,home
4,"Jokic, Nikola",NA,NaN,den,12.0,11,8,NaN,2,5,2,4-7,2-2,25:02,2020-12-16,por,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,"Metu, Chimezie",C,"$3,500",sac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2021-01-06,chi,home
363,"Okongwu, Onyeka",C,"$3,500",atl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2021-01-06,cha,home
364,"Pasecniks, Anzejs",C,"$3,500",was,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,0,2021-01-06,phi,away
365,"Richards, Nick",C,"$3,500",cha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2021-01-06,atl,away


In [31]:
for i in all_sals.points:
    try:
        float(i)
    except:
        print(i)

In [32]:
all_sals[all_sals.points=='']

,name,position,salary,team,points,rebounds,assists,steals,blocks,tos,threes,fg,ft,minutes,date,opponent,home_away
113,"Hampton, R.J.",PG,"$3,500",den,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2020-12-23,sac,home
60,"Hampton, R.J.",PG,"$3,500",den,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2020-12-25,lac,home
39,"Hampton, R.J.",PG,"$3,500",den,,1,NaN,NaN,NaN,NaN,NaN,1-1,NaN,4:07,2020-12-28,hou,home
88,"Hampton, R.J.",PG,"$3,500",den,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:24,2020-12-29,sac,away
85,"Hampton, R.J.",PG,"$3,500",den,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:37,2021-01-01,pho,home
58,"Hampton, R.J.",PG,"$3,500",den,,1,NaN,NaN,NaN,1,NaN,0-1,NaN,00:55,2021-01-03,min,away
44,"Hampton, R.J.",PG,"$3,500",den,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:06,2021-01-05,min,home


In [34]:
all_sals['points'] = all_sals.points.replace('',np.nan).astype(float)

In [35]:
all_sals.sort_values(by='points',ascending=False).head(10)

,name,position,salary,team,points,rebounds,assists,steals,blocks,tos,threes,fg,ft,minutes,date,opponent,home_away
0,"Curry, Stephen",PG,"$9,800",gsw,62.0,5,4,NaN,NaN,5,8,18-31,18-19,36:28,2021-01-03,por,home
0,"Beal, Bradley",SG,"$10,000",was,60.0,7,,1,NaN,4,7,20-35,13-15,38:15,2021-01-06,phi,away
0,"Morant, Ja",PG,"$7,800",mem,44.0,2,,2,1,1,1,18-27,7-8,35:10,2020-12-23,sas,home
1,"McCollum, C.J.",SG,"$7,200",por,44.0,3,8,1,NaN,NaN,9,17-30,1-2,43:50,2020-12-26,hou,home
0,"Harden, James",SG,"$11,000",hou,44.0,4,17,1,NaN,4,6,12-22,14-16,43:16,2020-12-26,por,away
164,"Hayward, Gordon",SF,"$7,700",cha,44.0,7,2,1,1,1,4,15-25,10-12,39:07,2021-01-06,atl,away
139,"Antetokounmpo, Giannis",PF,"$11,400",mil,43.0,9,4,NaN,1,3,2,17-24,7-9,36:04,2021-01-04,det,home
89,"Brown, Jaylen",SF,"$8,000",bos,42.0,5,4,1,NaN,4,7,15-21,5-6,28:57,2020-12-30,mem,home
2,"Rozier, Terry",PG,"$4,600",cha,42.0,3,2,2,1,3,10,15-23,2-4,37:31,2020-12-23,cle,away
142,"Tatum, Jayson",PF,"$9,100",bos,40.0,6,2,NaN,2,1,5,11-19,13-13,31:41,2021-01-04,tor,away


In [36]:
all_sals.to_csv(f'{end.year}_salaries.csv',index=False)

# OLD VERSION

In [3]:
def get_site(date):
    response = requests.get(f'http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon={date.month}&day={date.day}&year={date.year}')
    return response

def get_text(response):
    return response.text

def get_soup(text):
    bs(text)
    
def get_names(text):
    return pd.Series([i[1:-4] for i in re.findall(r'\>\D*,\D*<\/a>',text)],name = 'name')

def get_salaries(text):
    return pd.Series(re.findall(r'\$\d*,\d*',text),name = 'salary')


def get_df(date):
    response = get_site(date)
    text = get_text(response)
    names = get_names(text)
    salaries = get_salaries(text)
    dates = pd.Series([date]*len(names),name = 'date')
    df = pd.concat([dates,names,salaries],axis=1)
    return df
    
def all_dates(start_date, end_date, verbose=False, to_csv = False):
    date_range = pd.date_range(start_date, end_date)
    all_salaries = pd.DataFrame()
    for i in date_range:
        if verbose:
            print(i)
        df = get_df(i)
        all_salaries = all_salaries.append(df)
    if to_csv:
        all_salaries.to_csv(f'~/Desktop/{i.year}_salaries.csv',index=False)
    print('complete')
    
    
    return all_salaries

In [4]:
start = date(2016,10,1)
end = date(2017,9,1)

salary_2020 = all_dates(start,end,verbose=False,to_csv=True)

complete


In [6]:
salary_2020[salary_2020.date > datetime(2019,10,22)]

,date,name,salary
0,2019-10-23,"Irving, Kyrie","$9,000"
1,2019-10-23,"Mitchell, Donovan","$8,400"
2,2019-10-23,"Doncic, Luka","$9,900"
3,2019-10-23,"Lillard, Damian","$9,500"
4,2019-10-23,"Beal, Bradley","$9,800"
...,...,...,...
62,2020-09-01,"Bradley, Tony","$3,600"
63,2020-09-01,"Poirier, Vincent","$3,500"
64,2020-09-01,"Kanter, Enes","$4,000"
65,2020-09-01,"Bol, Bol","$3,800"
